# Trabalho 2 - Erikson Ferreira

## Importações

In [438]:
# Manipulação de dados
import numpy as np
import pandas as pd

# Geração de números aleatórios
import random

In [439]:
# Geração de gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import math

## Funções Auxiliares

### Solução Aleatória

In [440]:
# Cria uma solucao inicial com as cidades em um ordem aleatoria

def solucao_aleatoria(tsp):
    cidades = list(tsp.keys())
    solucao = []

    # as 3 linhas abaixo não são estritamente necessarias, servem
    # apenas para fixar a primeira cidade da lista na solução
    cidade = cidades[0]
    solucao.append(cidade)
    cidades.remove(cidade)

    for _ in range(0,len(cidades)):
        #print(_, cidades, solucao)
        cidade = random.choice(cidades)

        solucao.append(cidade)
        cidades.remove(cidade)

    return solucao

### Calcula Custo

In [441]:
# Função Objetivo: calcula custo de uma dada solução.
# Obs: Neste caso do problema do caixeiro viajante (TSP problem),
# o custo é o comprimento da rota entre todas as cidades.
def calcula_custo(tsp, solucao):

    N = len(solucao)
    custo = 0

    for i in range(N):

        # Quando chegar na última cidade, será necessário
        # voltar para o início para adicionar o
        # comprimento da rota da última cidade
        # até a primeira cidade, fechando o ciclo.
        #
        # Por isso, a linha abaixo:
        k = (i+1) % N
        cidadeA = solucao[i]
        cidadeB = solucao[k]

        custo += tsp.loc[cidadeA, cidadeB]

        #print(tsp.loc[cidadeA, cidadeB], cidadeA,cidadeB)

    return custo

### Gera Vizinhos

Obs: a função `obtem_vizinhos` descrita abaixo foi gerada de forma simplificada, pois ela assume que todos os vizinhos possuem rota direta entre si. Isto tem caráter didático para simplifcar a solução. Observe que na prática isso nem sempre existe rotas diretas entre todas as cidades e, em tais casos, pode ser necessário modificar a função para corresponder a tais restrições.

In [442]:
# A partir de uma dada solução, gera diversas variações (vizinhos)
def gera_vizinhos(solucao):

    N = len(solucao)
    for i in range(1, N):       # deixa o primeiro fixo
        for j in range(i + 1, N):
            vizinho = solucao.copy()
            vizinho[i] = solucao[j]
            vizinho[j] = solucao[i]

            yield(vizinho)


### Seleciona Melhor Vizinho

In [443]:
def obtem_melhor_vizinho(tsp, solucao):
    melhor_custo = calcula_custo(tsp, solucao)
    melhor_vizinho = solucao

    for vizinho in gera_vizinhos(solucao):
        custo_atual = calcula_custo(tsp, vizinho)
        if custo_atual < melhor_custo:
            melhor_custo = custo_atual
            melhor_vizinho = vizinho

    return melhor_vizinho, melhor_custo


### Random-Walk - clássico

In [444]:
def obtem_vizinho_aleatorio(tsp, solucao):

    vizinhos = list(gera_vizinhos(solucao))

    aleatorio_vizinho  = random.choice(vizinhos)
    aleatorio_custo    = calcula_custo(tsp, aleatorio_vizinho)

    return aleatorio_vizinho, aleatorio_custo


In [445]:
def random_walk(tsp):
    solucao_inicial = solucao_aleatoria(tsp)

    atual_solucao, atual_custo = obtem_vizinho_aleatorio(tsp, solucao_inicial)

    for _ in range(30):
        atual_solucao, atual_custo = obtem_vizinho_aleatorio(tsp, atual_solucao)

    return atual_custo, atual_solucao

### Hill-Climbing - clássico

In [446]:
def hill_climbing(tsp):
    history = []
    # solucao inicial
    solucao_inicial = solucao_aleatoria(tsp)
    print(solucao_inicial)
    # melhor solucao ate o momento
    solucao_melhor, custo_melhor = obtem_melhor_vizinho(tsp, solucao_inicial)
    

    while True:

        # tenta obter um candidato melhor
        candidato_atual, custo_atual = obtem_melhor_vizinho(tsp, solucao_melhor)
        #print(custo_melhor, custo_atual)

        if custo_atual < custo_melhor:
            custo_melhor   = custo_atual
            solucao_melhor = candidato_atual
            history.append(solucao_melhor)
        else:
            history.append(candidato_atual)
            break   # custo nao melhorou, entao sai do while

    return custo_melhor, solucao_melhor, history

### Hill-Climbing - Restart

In [447]:
def hill_climbing_restart(tsp, restarts=1000):

    # solucao inicial
    history = []
    solucao_inicial = solucao_aleatoria(tsp)
    # melhor solucao ate o momento
    solucao_melhor, custo_melhor = obtem_melhor_vizinho(tsp, solucao_inicial)

    for _ in range(restarts):

        # tenta obter um candidato melhor
        candidato_atual, custo_atual = obtem_melhor_vizinho(tsp, solucao_melhor)
        

        if custo_atual < custo_melhor:
            custo_melhor   = custo_atual
            solucao_melhor = candidato_atual
            history.append(solucao_melhor)
        else:
            solucao_inicial = solucao_aleatoria(tsp)
            solucao_melhor, custo_melhor = obtem_melhor_vizinho(tsp, solucao_inicial)
            history.append(solucao_melhor)
            # break   # custo nao melhorou, entao sai do while

    return custo_melhor, solucao_melhor, history

### Simulated Annealing

In [448]:
def calculate_distance(city_a, city_b):
    return np.linalg.norm(city_a - city_b)

def total_distance(route, distance_matrix):
    total = 0
    for i in range(len(route) - 1):
        city_a = route[i]
        city_b = route[i + 1]
        total += distance_matrix[city_a, city_b]
    return total

def generate_neighbor(route):
    new_route = route.copy()
    index_a = random.randint(0, len(route) - 1)
    index_b = random.randint(0, len(route) - 1)
    new_route[index_a], new_route[index_b] = new_route[index_b], new_route[index_a]
    return new_route

def acceptance_probability(current_distance, new_distance, temperature):
    if new_distance < current_distance:
        return 1.0
    else:
        return math.exp((current_distance - new_distance) / temperature)

In [449]:
def simulated_annealing(cities, initial_temperature = 1000, cooling_rate=0.90, iterations=1000):
    
    cities = cities.to_numpy()
    num_cities = len(cities)
    distance_matrix = np.zeros((num_cities, num_cities))

    for i in range(num_cities):
        for j in range(num_cities):
            distance_matrix[i, j] = calculate_distance(cities[i], cities[j])

    current_route = np.arange(num_cities)
    best_route = current_route.copy()

    current_distance = total_distance(current_route, distance_matrix)
    best_distance = current_distance

    temperature = initial_temperature

    #-----------------------------------------------
    iteration_list = []
    best_distances = []
    distance_list  = []
    accept_p_list  = []
    temperat_list  = []
    #-----------------------------------------------

    for iteration in range(iterations):
        new_route = generate_neighbor(current_route)
        new_distance = total_distance(new_route, distance_matrix)

        acceptance_prob = acceptance_probability(current_distance, new_distance, temperature)

        #print(acceptance_prob)

        if random.random() < acceptance_prob:
            current_route = new_route
            current_distance = new_distance

        if new_distance < best_distance:
            best_route = new_route
            best_distance = new_distance

        temperature *= cooling_rate

        #-----------------------------------------------
        iteration_list += [iteration]
        best_distances += [best_distance]
        distance_list  += [current_distance]
        accept_p_list  += [acceptance_prob]
        temperat_list  += [temperature]

        #-----------------------------------------------
        

    # plt.show()
    
    best_route = ['A' + str(num) for num in best_route]
    history = [best_route,{
        'iteration_list': iteration_list,
        'best_distances': best_distance,
        'distance_list': current_distance,
        'accept_p_list': accept_p_list,
        'temperature': temperature
    }]
    return best_distance, best_route, history

### Genetic Algorithm

In [450]:

def calcular_distancia(cidade1, cidade2):
    return np.linalg.norm(cidade1 - cidade2)

def distancia_total(rota, matriz_distancias):
    num_cidades = len(rota)
    distancia_total = 0
    for i in range(num_cidades):
        cidade_atual = rota[i]
        proxima_cidade = rota[(i + 1) % num_cidades]
        distancia_total += matriz_distancias[cidade_atual, proxima_cidade]
    return distancia_total

def gerar_vizinho(rota):
    num_cidades = len(rota)
    nova_rota = rota.copy()
    indice1 = random.randint(0, num_cidades - 1)
    indice2 = random.randint(0, num_cidades - 1)
    nova_rota[indice1], nova_rota[indice2] = nova_rota[indice2], nova_rota[indice1]
    return nova_rota

def crossover_ox(pai1, pai2):
    num_genes = len(pai1)
    pontos_corte = sorted(random.sample(range(num_genes), 2))
    filho = [-1] * num_genes
    
    filho[pontos_corte[0]:pontos_corte[1]+1] = pai1[pontos_corte[0]:pontos_corte[1]+1]
    
    j = (pontos_corte[1] + 1) % num_genes
    for i in range(num_genes):
        if pai2[i] not in filho:
            filho[j] = pai2[i]
            j = (j + 1) % num_genes
    
    return filho

def probabilidade_aceitacao(distancia_atual, nova_distancia, temperatura):
    if nova_distancia < distancia_atual:
        return 1.0
    return math.exp((distancia_atual - nova_distancia) / temperatura)

def algoritmo_genetico(cidades):
    tamanho_populacao=100
    num_geracoes=1000
    history = []
    cidades = cidades.to_numpy()
    num_cidades = len(cidades)
    matriz_distancias = np.zeros((num_cidades, num_cidades))

    for i in range(num_cidades):
        for j in range(num_cidades):
            matriz_distancias[i, j] = calcular_distancia(cidades[i], cidades[j])

    populacao = [np.random.permutation(num_cidades) for _ in range(tamanho_populacao)]

    melhor_rota = populacao[0]
    melhor_distancia = distancia_total(melhor_rota, matriz_distancias)

    for _ in range(num_geracoes):
        descendentes = []
        for _ in range(tamanho_populacao):
            pai1, pai2 = random.choices(populacao, k=2)
            filho = crossover_ox(pai1, pai2)
            descendentes.append(filho)

        populacao = descendentes

        for rota in populacao:
            distancia = distancia_total(rota, matriz_distancias)
            if distancia < melhor_distancia:
                melhor_rota = rota
                melhor_distancia = distancia
                history.append(melhor_rota)

    melhor_rota = ['A' + str(num) for num in melhor_rota]
    return melhor_distancia, melhor_rota, history





### Cálculo da Matriz de Distâncias

In [451]:
from math import sqrt

In [452]:
# distancia Euclidiana entre dois pontos
def distancia(x1,y1,x2,y2):
    dx = x2 - x1
    dy = y2 - y1
    return sqrt(dx**2 + dy**2)

In [453]:
# Calcula matriz de distancias.
#
# OBS:  Não é estritamente necessario calculá-las a priori.
#       Foi feito assim apenas para fins didáticos.
#       Ao invés, as distâncias podem ser calculadas sob demanda.

def gera_matriz_distancias(Coordenadas):

    n_cidades = len(Coordenadas)
    dist = np.zeros((n_cidades,n_cidades), dtype=float)

    for i in range(0, n_cidades):
        for j in range(i+1, n_cidades):
            x1,y1 = Coordenadas.iloc[i]
            x2,y2 = Coordenadas.iloc[j]

            dist[i,j] = distancia(x1,y1,x2,y2)
            dist[j,i] = dist[i,j]

    return dist

### Gerador de Problemas Aleatórios

In [454]:
# Gera aleatoriamente as coordenadas de N cidades.
# Obs: esta informação geralmente é fornecida como entrada do problema.

def gera_coordenadas_aleatorias(n_cidades):
    minimo = 10
    maximo = 90
    escala = (maximo-minimo)-1

    # gera n coordenadas (x,y) aleatorias entre [min, max]
    X = minimo + escala * np.random.rand(n_cidades)
    Y = minimo + escala * np.random.rand(n_cidades)
    coordenadas = {'X':X, 'Y': Y}

    cidades = ['A'+str(i) for i in range(n_cidades)]

    df_cidades = pd.DataFrame(coordenadas, index=cidades)
    df_cidades.index.name = 'CIDADE'

    return df_cidades


In [455]:
# Recebe uma lista com as coordenadas reais de uma cidade e
# gera uma matriz de distancias entre as cidades.
# Obs: a matriz é simetrica e com diagonal nula
def gera_problema_tsp(df_cidades):
    # nomes ficticios das cidades
    cidades = df_cidades.index

    # calcula matriz de distancias
    distancias = gera_matriz_distancias(df_cidades)

    # cria estrutura para armazena as distâncias entre todas as cidades
    tsp = pd.DataFrame(distancias, columns=cidades, index=cidades)

    return tsp

### Plota Rotas

In [456]:
# Plota a solução do roteamento das cidades
# usando a biblioteca PLOTLY
def plota_rotas(df_cidades, ordem_cidades):
    df_solucao = df_cidades.copy()
    df_solucao = df_solucao.reindex(ordem_cidades)

    X = df_solucao['X']
    Y = df_solucao['Y']
    cidades = list(df_solucao.index)

    # cria objeto gráfico
    fig = go.Figure()

    fig.update_layout(autosize=False, width=500, height=500, showlegend=False)

    # gera linhas com as rotas da primeira ate a ultima cidade
    fig.add_trace(go.Scatter(x=X, y=Y,
                             text=cidades, textposition='bottom center',
                             mode='lines+markers+text',
                             name=''))

    # acrescenta linha da última para a primeira para fechar o ciclo
    fig.add_trace(go.Scatter(x=X.iloc[[-1,0]], y=Y.iloc[[-1,0]],
                             mode='lines+markers', name=''))

    fig.show()

### Boxplots

In [457]:
def boxplot_sorted(df, rot=90, figsize=(12,6), fontsize=20):
    df2 = df.T
    meds = df2.median().sort_values(ascending=False)
    axes = df2[meds.index].boxplot(figsize=figsize, rot=rot, fontsize=fontsize,
                                   boxprops=dict(linewidth=4, color='cornflowerblue'),
                                   whiskerprops=dict(linewidth=4, color='cornflowerblue'),
                                   medianprops=dict(linewidth=4, color='firebrick'),
                                   capprops=dict(linewidth=4, color='cornflowerblue'),
                                   flierprops=dict(marker='o', markerfacecolor='dimgray',
                                        markersize=12, markeredgecolor='black'),
                                   return_type="axes")

    axes.set_title("Cost of Algorithms", fontsize=fontsize)

## Execução

### Executa 1 vez

In [458]:
# Simula a criação de N cidades
# com suas respectivas distâncias

n_cidades=10
df_coordenadas = gera_coordenadas_aleatorias(n_cidades)
df_coordenadas

,X,Y
CIDADE,,
A0,15.949499,41.044902
A1,59.332326,31.904361
A2,44.759579,76.966998
A3,70.378545,29.723792
A4,54.979947,47.049359
A5,80.417533,86.478742
A6,72.570100,46.915417
A7,84.931852,65.041568
A8,39.518028,54.784007


In [459]:
# solucao = ['A'+str(i) for i in range(n_cidades)]
# solucao
# plota_rotas(df_coordenadas, solucao)

In [460]:
# print(df_coordenadas.to_numpy())

In [461]:

custo, solucao, history = algoritmo_genetico(tsp)


print("Melhor solução encontrada:")
print(solucao)
print("Distância total da rota:", custo)
print(f'{custo:7.3f}    {solucao}')

# plota_rotas(df_coordenadas, solucao)

Melhor solução encontrada:
['A12', 'A0', 'A13', 'A17', 'A8', 'A18', 'A5', 'A1', 'A15', 'A4', 'A3', 'A7', 'A11', 'A2', 'A10', 'A6', 'A9', 'A14', 'A16', 'A19']
Distância total da rota: 1505.1233881816781
1505.123    ['A12', 'A0', 'A13', 'A17', 'A8', 'A18', 'A5', 'A1', 'A15', 'A4', 'A3', 'A7', 'A11', 'A2', 'A10', 'A6', 'A9', 'A14', 'A16', 'A19']


In [462]:

# # num_cities = 20
# # population_size = 100
# # num_generations = 1000

# best_solution, best_value = genetic_algorithm(df_coordenadas)

# print("Melhor solução encontrada:")
# print(best_solution)
# print("Distância total da rota:", best_value)

In [463]:
# solucao = solucao_aleatoria(tsp)
# print(solucao)
# plota_rotas(df_coordenadas, solucao)

In [ ]:
# # busca local da melhor solução e o seu custo
# custo, solucao = simulated_annealing(cities=tsp)
# # print(custo, solucao)
# print(f'{custo:7.3f}    {solucao}')

# plota_rotas(df_coordenadas, solucao)

In [ ]:
# # busca local da melhor solução e o seu custo
# custo, solucao = hill_climbing_restart(tsp,10)

# print(f'{custo:7.3f}    {solucao}')

# plota_rotas(df_coordenadas, solucao)

In [ ]:
# for _ in range(10):
#     custo, solucao = hill_climbing(tsp)

#     print(f'{custo:7.3f}    {solucao}')

#     plota_rotas(df_coordenadas, solucao)

### Executa N vezes - SIMPLES

In [ ]:
# # Executando várias vezes de forma simples

# n_vezes = 30

# for _ in range(n_vezes):

#     #solucao, custo = random_walk(tsp)
#     custo, solucao = hill_climbing(tsp)
#     print(f'{custo:7.3f}, {solucao}')

Observe que há uma certa variabilidade na soluções acima. Isto se deve à característica estocástica do algoritmo de solução.

---

### Executa N vezes - ESTRUTURADA com DataFrame

A seguir, é apresentada uma forma mais estruturada de se rodar várias vezes usando a estrutura de dados **`DataFrame`** para armazenar os resultados e permitir visualização de box-plots

In [ ]:
# Cria estruta de dados (DataFrame) para armazenar vários resultados
# diferentes e visualizá-los através de estatísticas

def cria_df_custos(algoritmos, n_vezes):

    nomes_algoritmos  = algoritmos.keys()

    n_lin = len(nomes_algoritmos)
    n_col = n_vezes

    df_results = pd.DataFrame(np.zeros((n_lin,n_col)),
                              index=nomes_algoritmos)
    df_results.index.name='ALGORITMO'

    return df_results

In [ ]:
# Executa N vezes para gerar estatísticas da variável custo
def executa_n_vezes(tsp, algoritmos, n_vezes):

    # Cria DataFrame para armazenar os resultados
    df_custo = cria_df_custos(algoritmos, n_vezes)
    df_history = pd.DataFrame(columns=algoritmos.keys())
    df_simulated_annealing = pd.DataFrame()
    for algoritmo, funcao_algoritmo in algoritmos.items():

        print(algoritmo)

        for i in range(n_vezes):
            df = None
            # if algoritmo != 'Genetic Algorithm':
            custo, solucao, historico = funcao_algoritmo(tsp)
            if len(historico)==2:
               
                df  = pd.DataFrame([historico[1]], columns=historico[1].keys())
                df_simulated_annealing = pd.concat([df_simulated_annealing, df], axis =0).reset_index(drop=True)
                # try:
                #     df_simulated_annealing.append(historico[1], ignore_index=True) 
                # except:
                #     print('error')   
            df_custo.loc[algoritmo,i] = custo
     
            # else:
                # custo, solucao = funcao_algoritmo(df_coord)
                # df_custo.loc[algoritmo,i] = custo

            print(f'{custo:10.3f}  {solucao}')

    return df_custo, df_history, df_simulated_annealing

In [ ]:
# Dicionario com Nomes dos modelos e suas respectivas variantes
# Tuple: (Algoritmo, Variante): funcao_algoritmo
algoritmos = {
    
    'Hill-Climbing': hill_climbing,
    'Hill-Climbing Restart': hill_climbing_restart,
    'Simulated Annealing': simulated_annealing,
    'Genetic Algorithm': algoritmo_genetico,
}

#### PROBLEMA GERADO ALEATORIAMENTE

In [ ]:
###################################
# PROBLEMA GERADO ALEATORIAMENTE  #
###################################

# cria instancia do problema com n cidades

n_cidades=20
df_coordenadas = gera_coordenadas_aleatorias(n_cidades)

tsp = gera_problema_tsp(df_coordenadas)

# numero de vezes que executará cada algoritmo
n_vezes = 30

# Executa N vezes para gerar estatísticas da variável custo
df_custo, df_history, df_simulated_annealing = executa_n_vezes(tsp, algoritmos, n_vezes)

Simulated Annealing
<class 'dict'>
histórico ['A9', 'A6', 'A0', 'A12', 'A13', 'A8', 'A16', 'A18', 'A11', 'A17', 'A10', 'A2', 'A7', 'A4', 'A3', 'A5', 'A15', 'A1', 'A19', 'A14']
  1216.715  ['A9', 'A6', 'A0', 'A12', 'A13', 'A8', 'A16', 'A18', 'A11', 'A17', 'A10', 'A2', 'A7', 'A4', 'A3', 'A5', 'A15', 'A1', 'A19', 'A14']
<class 'dict'>
histórico ['A10', 'A2', 'A3', 'A4', 'A7', 'A17', 'A12', 'A11', 'A18', 'A5', 'A9', 'A6', 'A0', 'A13', 'A8', 'A14', 'A19', 'A1', 'A15', 'A16']
  1281.703  ['A10', 'A2', 'A3', 'A4', 'A7', 'A17', 'A12', 'A11', 'A18', 'A5', 'A9', 'A6', 'A0', 'A13', 'A8', 'A14', 'A19', 'A1', 'A15', 'A16']
<class 'dict'>
histórico ['A18', 'A5', 'A19', 'A1', 'A15', 'A16', 'A11', 'A7', 'A17', 'A12', 'A0', 'A13', 'A8', 'A14', 'A9', 'A6', 'A10', 'A2', 'A4', 'A3']
  1265.973  ['A18', 'A5', 'A19', 'A1', 'A15', 'A16', 'A11', 'A7', 'A17', 'A12', 'A0', 'A13', 'A8', 'A14', 'A9', 'A6', 'A10', 'A2', 'A4', 'A3']
<class 'dict'>
histórico ['A18', 'A11', 'A3', 'A4', 'A7', 'A2', 'A10', 'A17', 'A12'

KeyboardInterrupt: 

##### Box Plots

In [ ]:
df_custo_T = df_custo.T
df_custo_T

ALGORITMO,Simulated Annealing,Hill-Climbing,Hill-Climbing Restart,Genetic Algorithm
0,1689.611705,337.022627,363.949880,1454.493861
1,1705.849014,415.644178,351.475993,1352.544614
2,1686.932332,330.655903,367.233011,1403.506989
3,1716.112661,381.029887,377.450896,1389.755757
4,1866.255385,316.553062,392.193016,1397.644836
...,...,...,...,...
95,1537.649249,370.447805,349.565291,1394.938240
96,1866.255385,351.024767,399.210783,1465.682575
97,1866.255385,362.366527,347.735688,1465.313742
98,1784.449236,328.481867,387.847661,1248.318666


In [ ]:
px.box(df_custo_T, points='all', y=df_custo_T.columns.tolist()).update_layout(
    xaxis_title="Algoritmo", yaxis_title="Custo", title="Custo por Algoritmo")

In [ ]:
boxplot_sorted(df_custo, rot=90, figsize=(12,6), fontsize=20)

Observe no gráfico acima que a variabilidade do Hill-climbing é bem menor que a do Random-Walk.

Como você justifica isso?

In [ ]:
df_custo.T.describe()

ALGORITMO,Simulated Annealing,Hill-Climbing,Hill-Climbing Restart,Genetic Algorithm
count,100.000000,100.000000,100.000000,100.000000
mean,2070.261380,359.775717,427.835944,1462.107578
std,128.630532,33.458712,113.648968,48.938673
min,1731.485283,299.825597,288.834137,1283.675813
25%,1980.314923,335.125779,366.073866,1437.771657
50%,2095.282855,357.101881,390.353261,1467.920896
75%,2167.545128,381.212907,423.701700,1494.397959
max,2320.116081,446.915143,774.303647,1553.514898


### Plota a rota da melhor solução obtida

In [ ]:
df_custo.T

ALGORITMO,Simulated Annealing,Hill-Climbing,Hill-Climbing Restart,Genetic Algorithm
0,2035.524518,351.183094,385.100074,1495.418659
1,2154.021676,385.717125,378.690339,1492.849401
2,2015.504976,365.981696,396.908251,1443.681556
3,2105.774362,396.039375,413.330833,1466.011477
4,2068.277514,343.915509,373.782650,1441.869613
...,...,...,...,...
95,1988.819925,429.205627,542.440408,1537.176402
96,1858.843830,390.972422,428.919443,1283.675813
97,1979.757466,362.080054,423.076317,1432.258187
98,2094.784707,343.016789,730.841940,1495.925150


In [ ]:
df_simulated_annealing

,iteration_list,best_distances,distance_list,accept_p_list,temperature
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2035.524518,2335.433071,"[0.9998258612712249, 1.0, 1.0, 1.0, 1.0, 0.948...",904.382075
1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2154.021676,2154.021676,"[0.9834283724797221, 1.0, 1.0, 0.9428473915639...",904.382075
2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2015.504976,2506.373850,"[1.0, 0.8256734532793585, 1.0, 1.0, 1.0, 1.0, ...",904.382075
3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2105.774362,2319.936419,"[1.0, 0.9926845029922797, 1.0, 1.0, 0.91579572...",904.382075
4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2068.277514,2068.277514,"[0.9956967240201055, 0.896838132291118, 1.0, 1...",904.382075
...,...,...,...,...,...
95,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1988.819925,2435.706898,"[1.0, 1.0, 0.9460873069787443, 1.0, 0.99628223...",904.382075
96,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1858.843830,1858.843830,"[0.9494343582963197, 1.0, 1.0, 1.0, 1.0, 1.0, ...",904.382075
97,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",1979.757466,2191.479683,"[1.0, 1.0, 1.0, 1.0, 0.9390166523761565, 1.0, ...",904.382075
98,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",2094.784707,2190.293542,"[0.985976713484964, 0.9489368615194189, 1.0, 0...",904.382075
